## **Install Packaegs**

In [1]:
!pip install wikipedia
!pip install paramiko
!pip install pprint

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=31bf12df5f6a61ffcd90b59c1b54e47e756bf09ad2d37e9185d632663240b475
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 215kB 11.8MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 962kB 39.3MB/s 
     |████████████████████████████████| 2.6MB 53.1MB/s 
ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


## **Import required libraries**

In [2]:
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel 
import wikipedia
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

import pandas as pd
import pickle
import pprint

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### **Read pickle - This consist of the tweets dataframe. Replace the below with your working tweets dataframe if needed**

This scoring system is tested on a sample tweet dataset

In [4]:
# Import dataframe
tweets_dataset = pd.read_pickle("/content/tweets_cleansed")

In [5]:
tweets_dataset

,dates,favorite_count,hashtags,id_str,retweet_count,text,urls,user_description,user_followers_count,user_friends_count,user_id,user_location,user_name,user_screen_name,user_url,text_Cleansed,text_Cleansed_ngrams
0,2019-04-02 11:48:17,3,"[{'text': 'Super', 'indices': [3, 9]}, {'text'...",1113045483038130176,1,🥗🍗🍜#Super Easy Ways To Learn Everything About ...,"[{'url': 'https://t.co/ipP2RGvvnI', 'expanded_...",keep you up to date with all #SocialMedia #Rec...,21993,8886,95063624,,Maher Mashaal,mymws,https://t.co/VpKDQDwsfU,"[, super, easy, way, learn, everything, preven...",super easy way learn everything preventing he...
1,2019-04-02 10:15:04,2,"[{'text': 'highfiber', 'indices': [9, 19]}, {'...",1113022023893159936,0,Choose a #highfiber diet for a lower risk of #...,"[{'url': 'https://t.co/41kPPgXfio', 'expanded_...","Medical writer, publisher of nutrient counter ...",9151,6874,2783782729,"Marlow, UK",Dr Lindy van den Berghe,250HealthyFoods,https://t.co/q5Bv4yAajl,"[choose, highfiber, diet, lower, risk, heart, ...",choose highfiber diet lower risk heart attack ...
4,2019-04-04 07:11:51,0,"[{'text': 'summerhealth', 'indices': [0, 13]},...",1113700691586031616,0,#summerhealth\n\nCommon digestive problems dur...,[],Dr. Rela Institute and Medical Centre is a 450...,82,26,1036903613946245122,"Chennai, India",Dr. Rela Institute & Medical Centre,Rela_Institute,https://t.co/rrdjvhSazN,"[summerhealth, common, digestive, problem, sum...",summerhealth common digestive problem summer s...
5,2019-04-04 07:11:35,0,"[{'text': 'diet', 'indices': [81, 86]}, {'text...",1113700626846949376,110,RT @TheLancet: One in five deaths (11 million)...,[],/ psychiatrist / neuroscience / sceptic / blet...,832,2737,1211649024,"Glasgow, Scotland",Robert Hunter,BobHunterMD,None,"[rt, thelancet, one, five, death, 11, million,...",rt thelancet one five death 11 million 2017 as...
6,2019-04-04 07:09:20,0,"[{'text': 'ICTUSnet', 'indices': [58, 67]}, {'...",1113700060741738496,0,.@Patgompic kicks off the 3rd General Assembly...,[],@Sudoe5 project | Excellence R&D network towar...,71,92,989435649693536257,Europe,ICTUSnet,ICTUSnet,https://t.co/oO6e3UVUDP,"[patgompic, kick, 3rd, general, assembly, meet...",patgompic kick 3rd general assembly meeting ic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10766,2019-04-01 01:12:29,0,"[{'text': 'BloodPressure', 'indices': [73, 87]...",1112523090351648768,8,RT @healthyojas: Blood Pressure &amp; Heart Ra...,"[{'url': 'https://t.co/74FfmXn7L2', 'expanded_...",,284,1414,1048197839279415296,,Manjula,Manjula09540220,None,"[rt, healthyojas, blood, pressure, amp, heart,...",rt healthyojas blood pressure amp heart rate 7...
10767,2019-04-01 01:09:16,0,"[{'text': 'BloodPressure', 'indices': [73, 87]...",1112522282319015936,8,RT @healthyojas: Blood Pressure &amp; Heart Ra...,"[{'url': 'https://t.co/74FfmXn7L2', 'expanded_...","Tips on #Digestion, Digestive disorders (#GERD...",14729,11496,416871524,,digestionsystem,digestionsystem,http://t.co/HhmMUOoOS7,"[rt, healthyojas, blood, pressure, amp, heart,...",rt healthyojas blood pressure amp heart rate 7...
10768,2019-04-01 01:03:16,0,"[{'text': 'BloodPressure', 'indices': [52, 66]...",1112520774156312576,7,RT @healthyojas: Heart Rate https://t.co/u5pAk...,"[{'url': 'https://t.co/u5pAkKlNyo', 'expanded_...","Tips on #Digestion, Digestive disorders (#GERD...",14729,11496,416871524,,digestionsystem,digestionsystem,http://t.co/HhmMUOoOS7,"[rt, healthyojas, heart, rate, u5pakklnyo, blo...",rt healthyojas heart rate u5pakklnyo bloodpres...
10769,2019-04-01 00:51:09,5,"[{'text': 'hypertension', 'indices': [201, 214]}]",1112517723672596480,0,@brandimwynne @smac121 @AtulPathak31 @thebyrdl...,[],"Senior Hospital Scientist, Head, Cardiovascula...",6629,2714,4700114665,"Sydney, New South Wales",Dr Anastasia Susie Mihailidou FAHA FCSANZ,AnastasiaSMihai,None,"[brandimwynne, smac121, atulpathak31, thebyrdl...",brandimwynne smac121 atulpathak31 thebyrdlab s...


### **Creates the scoring for the tweets**

In [6]:
# Fuction to generate frequency dictionary from wikipedia contents
def get_freq_dict(search):
    wiki = ""
    for query in search:
        wiki += wikipedia.page(query).content

    wiki = strip_punctuation(wiki) #remove punctuations
    wiki = wiki.lower() #convert to lower case
    wiki = remove_stopwords(wiki) #remove stop words
    words_in_text = wiki.split()

    cnt = Counter()
    for word in words_in_text:
      cnt[word] += 1
      total_word_count = float(sum(cnt.values()))
      cnt[word] = float(cnt[word]/total_word_count)

    freq_dict = dict(cnt)
    return freq_dict
  
# Fuction to match words from dictionary with words and tweets and generate tweet relevence score
def match_tweets(text):
    count = 0
    sum = 0
#     line = text.split()
    for word in text:
        count += 1
        if word in freq_dict:
            sum = sum + freq_dict[word] + 1
        else:
            sum -= 0.25
    if count != 0:
        return sum/count

search = ["cardiovascular disease"] # Add more words here
freq_dict = get_freq_dict(search)

tweets_dataset['Score'] = tweets_dataset['text_Cleansed'].apply(match_tweets)
tweets_dataset.to_csv("tweets_dataset.csv")

### Filter out any tweets that have a negative overall scoring and remove any tweets entries that are re-tweets of others

In [7]:
useful_tweets = tweets_dataset.loc[(tweets_dataset['Score'] > 0.)]
useful_tweets = useful_tweets[~useful_tweets['text_Cleansed_ngrams'].str.contains('rt')]

In [ ]:
useful_tweets

,dates,favorite_count,hashtags,id_str,retweet_count,text,urls,user_description,user_followers_count,user_friends_count,user_id,user_location,user_name,user_screen_name,user_url,text_Cleansed,text_Cleansed_ngrams,Score
11,2019-04-04 07:00:45,0,"[{'text': 'rehab', 'indices': [78, 84]}, {'tex...",1113697900691972096,0,"Get the April issue (2,2019) of Health is Your...","[{'url': 'https://t.co/Noc4OORBTY', 'expanded_...",Self Help Neurological magazine in Print on Am...,356,202,2490731019,UK,Health isYour Wealth,HiYWMag,https://t.co/6TOx36fGLn,"[get, april, issue, 22019, health, wealth, mag...",get april issue 22019 health wealth magazine e...,0.014536
12,2019-04-04 07:00:39,0,"[{'text': 'strokeprevention', 'indices': [166,...",1113697872703311872,0,Interesting new research published shows patie...,"[{'url': 'https://t.co/bCeK8kEFeu', 'expanded_...",Exploring the world of connected health and ne...,164,522,4883269864,"London, England",kim ramessa,kimramessa77,https://t.co/eNwinieraA,"[interesting, new, research, published, show, ...",interesting new research published show patien...,0.175642
13,2019-04-04 06:59:53,0,"[{'text': 'CVD', 'indices': [9, 13]}, {'text':...",1113697682634231808,0,Diet and #CVD risk. The high price of a poor d...,"[{'url': 'https://t.co/AjyGnNwLoC', 'expanded_...",Nutrition & #Health for all. BSc Hons & MSc Sp...,2495,2126,51484172,"Wirral, England",MoyraCosNutrition,MoyraCosgrove,https://t.co/hFkeVdKsi1,"[diet, cvd, risk, high, price, poor, diet, hea...",diet cvd risk high price poor diet health nh a...,0.644767
47,2019-04-04 06:07:38,0,"[{'text': 'Young', 'indices': [59, 65]}, {'tex...",1113684531813470208,0,(Psychostimulants play a major role in fatal s...,"[{'url': 'https://t.co/KTg34IDDB0', 'expanded_...","Talent Scout, Human Resources Management, Tale...",4297,4986,285544157,"Asia Pacific, EMEA and US",Christian Hoffeldt,hoffeldtcom,http://t.co/OxDmAniagL,"[psychostimulants, play, major, role, fatal, s...",psychostimulants play major role fatal stroke ...,0.358109
56,2019-04-04 06:00:00,0,"[{'text': 'stroke', 'indices': [118, 125]}, {'...",1113682611342327808,0,NHS Choices brings you Health Today Radio! wit...,"[{'url': 'https://t.co/4ZzliEQ9Ge', 'expanded_...",#UKHealthRadio Health & Wellness Radio in the ...,42385,28671,569242704,London UK and the world,UK Health Radio™,ukhealthradio,https://t.co/JTfT2VXQI9,"[nh, choice, brings, health, today, radio, dom...",nh choice brings health today radio dominic ar...,0.083629
89,2019-04-04 04:57:12,0,"[{'text': 'Stroke', 'indices': [14, 21]}, {'te...",1113666808979042304,0,Singing Helps #Stroke Patients suffering from ...,"[{'url': 'https://t.co/ckmFxa8NcV', 'expanded_...","To educate and inform people about the facts, ...",3145,650,902600646,Kansas City,Aphasia Hope,AphasiaAnswers,http://t.co/x5vdmCXoQE,"[singing, help, stroke, patient, suffering, ap...",singing help stroke patient suffering aphasia ...,0.115252
181,2019-04-04 01:55:04,0,"[{'text': 'Stroke', 'indices': [0, 7]}, {'text...",1113620970747641856,0,"#Stroke position in Grand Rapids, MI https://t...","[{'url': 'https://t.co/LNYu5laSRf', 'expanded_...",Doctor Chicago is “Your Local Chicagoland Phys...,20,95,984447161810202625,"Naperville, IL",Doctor Chicago,DoctorChicagoMD,https://t.co/NTsoUZV08F,"[stroke, position, grand, rapid, mi, lnyu5lasr...",stroke position grand rapid mi lnyu5lasrf drch...,0.005210
183,2019-04-04 01:48:52,0,"[{'text': 'VirginiaTech', 'indices': [0, 13]},...",1113619411049504768,0,#VirginiaTech Launches the First Clinical #Tri...,"[{'url': 'https://t.co/fKXqDDPlgG', 'expanded_...",TrialSite News is the only #digital media dedi...,381,2137,1002361483169226752,"Salt Lake City, Utah",TrialSite News,TrialsiteN,https://t.co/LRKGg0J5xs,"[virginiatech, launch, first, clinical, trial,...",virginiatech launch first clinical trial rehab...,0.090435
224,2019-04-04 00:41:00,10,"[{'text': 'study', 'indices': [79, 85]}, {'tex...",1113602333907222528,1,Happy to host Dr Marc Roig at McMaste